In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics

import os

In [ ]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

# Output classes to learn how to classify
LABELS = [
    "WALKING", 
    "WALKING_UPSTAIRS", 
    "WALKING_DOWNSTAIRS", 
    "SITTING", 
    "STANDING", 
    "LAYING"
] 

In [ ]:
from re import A
text_file = open("/data/WISDM_ar_v1.1_raw.txt")
lines = text_file.read().split(';')
text_file.close()

x = np.ndarray(shape=(1098208,3))
y = list()

for i in range(1098208):
  a = lines[i].split(',')
  a = np.array(a) 
  # print(a[3:6])
  x[i,:] = a[3:6]
  y.append(a[1])
  


/content
sample_data
/data
download_dataset.py  source.txt

Downloading...
--2022-11-04 10:46:10--  https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60999314 (58M) [application/x-httpd-php]
Saving to: ‘UCI HAR Dataset.zip’

UCI HAR Dataset.zip 100%[===================>]  58.17M  45.1MB/s    in 1.3s    

2022-11-04 10:46:11 (45.1 MB/s) - ‘UCI HAR Dataset.zip’ saved [60999314/60999314]


Extracting...
Extracting successfully done to /data/UCI HAR Dataset.
/data
 download_dataset.py   source.txt	 'UCI HAR Dataset.zip'
 __MACOSX	      'UCI HAR Dataset'
/
bin	 datalab  lib	 mnt			   python-apt  srv    usr
boot	 dev	  lib32  NGC-DL-CONTAINER-LICENSE  root        sys    var
content  etc	  lib64  opt			   run	       tmp
data	 home	  media  proc

In [ ]:
import pandas as pd
pd.DataFrame(x).to_csv("/WISDM_x.csv")
pd.DataFrame(y).to_csv("/WISDM_y.csv")

In [ ]:
# y_prep = np.ndarray(shape=len(y))
# for i in range(len(y)):
#   if(y[i] == 'Walking'):
#     y_prep[i] = 1
#   elif(y[i] == "Jogging"):
#     y_prep[i] = 2
#   elif(y[i] == "Uptairs"):
#     y_prep[i] = 3
#   elif(y[i] == "Downstairs"):
#     y_prep[i] = 4
#   elif(y[i] == "Sitting"):
#     y_prep[i] = 5
#   elif(y[i] == "Standing"):
#     y_prep[i] = 6


In [ ]:
# X_train.shape


(7352, 128, 9)

In [ ]:
# y_train.shape

(7352, 1)

In [ ]:
# print(y_train)

[[4]
 [4]
 [4]
 ...
 [1]
 [1]
 [1]]


In [ ]:
# # Input Data 

# training_data_count = len(X_train)  # 7352 training series (with 50% overlap between each serie)
# test_data_count = len(X_test)  # 2947 testing series
# n_steps = len(X_train[0])  # 128 timesteps per series
# n_input = len(X_train[0][0])  # 9 input parameters per timestep


# # LSTM Neural Network's internal structure

# n_hidden = 32 # Hidden layer num of features
# n_classes = 6 # Total classes (should go up, or should go down)


# # Training 

# learning_rate = 0.0025
# lambda_loss_amount = 0.0015
# training_iters = training_data_count * 300  # Loop 300 times on the dataset
# batch_size = 1500
# display_iter = 30000  # To show test set accuracy during training


# # Some debugging info

# print("Some useful info to get an insight on dataset's shape and normalisation:")
# print("(X shape, y shape, every X's mean, every X's standard deviation)")
# print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
# print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(2947, 128, 9) (2947, 1) 0.09913992 0.39567086
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


In [ ]:
# def LSTM_RNN(_X, _weights, _biases):
#     # Function returns a tensorflow LSTM (RNN) artificial neural network from given parameters. 
#     # Moreover, two LSTM cells are stacked which adds deepness to the neural network. 
#     # Note, some code of this notebook is inspired from an slightly different 
#     # RNN architecture used on another dataset, some of the credits goes to 
#     # "aymericdamien" under the MIT license.

#     # (NOTE: This step could be greatly optimised by shaping the dataset once
#     # input shape: (batch_size, n_steps, n_input)
#     _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
#     # Reshape to prepare input to hidden activation
#     _X = tf.reshape(_X, [-1, n_input]) 
#     # new shape: (n_steps*batch_size, n_input)
    
#     # ReLU activation, thanks to Yu Zhao for adding this improvement here:
#     _X = tf.nn.relu(tf.matmul(_X, _weights['hidden']) + _biases['hidden'])
#     # Split data because rnn cell needs a list of inputs for the RNN inner loop
#     _X = tf.split(_X, n_steps, 0) 
#     # new shape: n_steps * (batch_size, n_hidden)

#     # Define two stacked LSTM cells (two recurrent layers deep) with tensorflow
#     lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
#     lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
#     lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
#     # Get LSTM cell output
#     outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)

#     # Get last time step's output feature for a "many-to-one" style classifier, 
#     # as in the image describing RNNs at the top of this page
#     lstm_last_output = outputs[-1]
    
#     # Linear activation
#     return tf.matmul(lstm_last_output, _weights['out']) + _biases['out']

# def extract_batch_size(_train, step, batch_size):
#     # Function to fetch a "batch_size" amount of data from "(X|y)_train" data. 
    
#     shape = list(_train.shape)
#     shape[0] = batch_size
#     batch_s = np.empty(shape)

#     for i in range(batch_size):
#         # Loop index
#         index = ((step-1)*batch_size + i) % len(_train)
#         batch_s[i] = _train[index] 

#     return batch_s

# def one_hot(y_, n_classes=n_classes):
#     # Function to encode neural one-hot output labels from number indexes 
#     # e.g.: 
#     # one_hot(y_=[[5], [0], [3]], n_classes=6):
#     #     return [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    
#     y_ = y_.reshape(len(y_))
#     return np.eye(n_classes)[np.array(y_, dtype=np.int32)]  # Returns FLOATS

NameError: ignored